<a href="https://colab.research.google.com/github/sukanya3970/CODSOFT/blob/main/new_Task2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os

def move_csv_files(file1_path, file2_path, destination_dir):
  """
  Moves two CSV files to the specified destination directory.

  Args:
    file1_path: Path to the first CSV file.
    file2_path: Path to the second CSV file.
    destination_dir: Path to the destination directory.
  """
  try:
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Move the files using shutil.move
    shutil.move(file1_path, destination_dir)
    shutil.move(file2_path, destination_dir)

    print(f"Files moved to '{destination_dir}' successfully.")

  except Exception as e:
    print(f"Error moving files: {e}")

# Example usage:
file1_path = "fraudTest.csv"
file2_path = "fraudTrain.csv"
destination_dir = "csv_files"  # Create a directory named 'csv_files'

move_csv_files(file1_path, file2_path, destination_dir)


Files moved to 'csv_files' successfully.


In [ ]:
import shutil
import os
import pandas as pd

def move_csv_files(file1_path, file2_path, destination_dir):
  """
  Moves two CSV files to the specified destination directory.

  Args:
    file1_path: Path to the first CSV file.
    file2_path: Path to the second CSV file.
    destination_dir: Path to the destination directory.
  """
  try:
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Move the files using shutil.move
    shutil.move(file1_path, destination_dir)
    shutil.move(file2_path, destination_dir)

    print(f"Files moved to '{destination_dir}' successfully.")

  except Exception as e:
    print(f"Error moving files: {e}")


def read_fraud_csv_files(destination_dir):
    """
    Reads two CSV files named "Fraud test.csv" and "Fraud Train.csv"
    from the specified directory into pandas DataFrames.

    Args:
        destination_dir: The directory where the CSV files are located.

    Returns:
        A tuple containing the two pandas DataFrames, or (None, None) if an error occurs.
    """
    try:
        # Construct the full file paths
        test_file_path = os.path.join(destination_dir, "fraudTest.csv")
        train_file_path = os.path.join(destination_dir, "fraudTrain.csv")

        # Read the CSV files into DataFrames
        df_test = pd.read_csv(test_file_path)
        df_train = pd.read_csv(train_file_path)
        return df_test, df_train
    except FileNotFoundError:
        print("Error: One or both of the CSV files were not found in the specified directory.")
        return None, None


# Example usage:
file1_path = "Fraud test.csv"
file2_path = "Fraud Train.csv"
destination_dir = "csv_files"  # Create a directory named 'csv_files'

# Move the files
move_csv_files(file1_path, file2_path, destination_dir)

# Read the files from the destination directory
df_test, df_train = read_fraud_csv_files(destination_dir)

if df_test is not None and df_train is not None:
    print("Fraud Test Data:")
    print(df_test.head())  # Print the first few rows of df_test
    print("\nFraud Train Data:")
    print(df_train.head())  # Print the first few rows of df_train
    df_train.isnull().sum()


Error moving files: [Errno 2] No such file or directory: 'Fraud test.csv'
Fraud Test Data:
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2020-06-21 12:14:25  2291163933867244   
1           1   2020-06-21 12:14:33  3573030041201292   
2           2   2020-06-21 12:14:53  3598215285024754   
3           3   2020-06-21 12:15:15  3591919803438423   
4           4   2020-06-21 12:15:17  3526826139003047   

                               merchant        category    amt   first  \
0                 fraud_Kirlin and Sons   personal_care   2.86    Jeff   
1                  fraud_Sporer-Keebler   personal_care  29.84  Joanne   
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28  Ashley   
3                     fraud_Haley Group        misc_pos  60.05   Brian   
4                 fraud_Johnston-Casper          travel   3.19  Nathan   

       last gender                       street  ...      lat      long  \
0   Elliott      M            351 Darlene Gree

In [ ]:
def data_pre(X):
    # These columns are presumably considered irrelevant for the analysis
    del_col=['merchant','first','last','street','zip','unix_time','Unnamed: 0','trans_num','cc_num']
    X.drop(columns=del_col,inplace=True)

    # Data Conversion
    # The trans_date_trans_time column is then used to create a new column
    # trans_date that contains only the date part in the 'YYYY-MM-DD' format.
    X['trans_date_trans_time']=pd.to_datetime(X['trans_date_trans_time'])
    X['trans_date']=X['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
    X['trans_date']=pd.to_datetime(X['trans_date'])
    X['dob']=pd.to_datetime(X['dob'])

    #Calculate Age of each transaction
    X["age"] = (X["trans_date"] - X["dob"]).dt.days //365
    X['trans_month']=X['trans_date'].dt.month
    X['trans_year']=X['trans_date'].dt.year
    # 'M' (presumably for males) is converted to 1, and other values are converted to 0.
    X['gender']=X['gender'].apply(lambda x : 1 if x=='M' else 0)
    X['gender']=X['gender'].astype(int)

    # The lat_dis and long_dis columns are created to calculate the absolute
    # differences between the latitude (lat) and merchant latitude (merch_lat)
    # as well as the longitude (long) and merchant longitude (merch_long).
    X['lat_dis']=abs(X['lat']-X['merch_lat'])
    X['long_dis']=abs(X['long']-X['merch_long'])

    # The function applies one-hot encoding to the category column to convert categorical variables
    # into binary indicators. This is achieved using pd.get_dummies.
    X=pd.get_dummies(X,columns=['category'])
    X=X.drop(columns=['city','trans_date_trans_time','state','job','merch_lat','merch_long','lat','long','dob','trans_date'])
    return X

    # The gender column is transformed to binary values where
    # 'M' (presumably for males) is converted to 1, and other values are converted to 0.


In [ ]:
train_df_pre=data_pre(train_df.copy())
train_df_pre.head()
test_df_pre=data_pre(test_df.copy())
test_df_pre.head()

NameError: name 'train_df' is not defined

In [ ]:
import shutil
import os
import pandas as pd

def move_csv_files(file1_path, file2_path, destination_dir):
    """
    Moves two CSV files to the specified destination directory.

    Args:
        file1_path: Path to the first CSV file.
        file2_path: Path to the second CSV file.
        destination_dir: Path to the destination directory.
    """
    try:
        # Create the destination directory if it doesn't exist
        os.makedirs(destination_dir, exist_ok=True)

        # Move the files using shutil.move
        shutil.move(file1_path, destination_dir)
        shutil.move(file2_path, destination_dir)

        print(f"Files moved to '{destination_dir}' successfully.")

    except Exception as e:
        print(f"Error moving files: {e}")


def read_fraud_csv_files(destination_dir):
    """
    Reads two CSV files named "Fraud test.csv" and "Fraud Train.csv"
    from the specified directory into pandas DataFrames.

    Args:
        destination_dir: The directory where the CSV files are located.

    Returns:
        A tuple containing the two pandas DataFrames, or (None, None) if an error occurs.
    """
    try:
        # Construct the full file paths
        test_file_path = os.path.join(destination_dir, "Fraud test.csv")
        train_file_path = os.path.join(destination_dir, "Fraud Train.csv")

        # Read the CSV files into DataFrames
        df_test = pd.read_csv(test_file_path)
        df_train = pd.read_csv(train_file_path)
        return df_test, df_train
    except FileNotFoundError:
        print("Error: One or both of the CSV files were not found in the specified directory.")
        return None, None


def data_pre(X):
    """
    Preprocesses the input DataFrame X.

    Args:
        X: The input DataFrame.

    Returns:
        The preprocessed DataFrame.
    """
    # These columns are presumably considered irrelevant for the analysis
    del_col = ['merchant', 'first', 'last', 'street', 'zip', 'unix_time',
               'Unnamed: 0', 'trans_num', 'cc_num']
    X.drop(columns=del_col, inplace=True)

    # Data Conversion
    X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time'])
    X['trans_date'] = X['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
    X['trans_date'] = pd.to_datetime(X['trans_date'])
    X['dob'] = pd.to_datetime(X['dob'])

    # Calculate Age of each transaction
    X["age"] = (X["trans_date"] - X["dob"]).dt.days // 365
    X['trans_month'] = X['trans_date'].dt.month
    X['trans_year'] = X['trans_date'].dt.year

    # Convert gender to binary (1 for 'M', 0 for others)
    X['gender'] = X['gender'].apply(lambda x: 1 if x == 'M' else 0)
    X['gender'] = X['gender'].astype(int)

    # Calculate latitude and longitude differences
    X['lat_dis'] = abs(X['lat'] - X['merch_lat'])
    X['long_dis'] = abs(X['long'] - X['merch_long'])

    # Apply one-hot encoding to the category column
    X = pd.get_dummies(X, columns=['category'])
    X = X.drop(columns=['city', 'trans_date_trans_time', 'state', 'job',
                        'merch_lat', 'merch_long', 'lat', 'long', 'dob',
                        'trans_date'])
    return X



In [ ]:
train_df_pre=data_pre(df_train.copy()) # Changed train_df to df_train
train_df_pre.head()
test_df_pre=data_pre(df_test.copy()) # Changed test_df to df_test
test_df_pre.head()

,amt,gender,city_pop,is_fraud,age,trans_month,trans_year,lat_dis,long_dis,category_entertainment,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,1,333497,0,52,6,2020,0.020491,0.265214,False,...,False,False,False,False,False,False,True,False,False,False
1,29.84,0,302,0,30,6,2020,0.870202,0.475569,False,...,False,False,False,False,False,False,True,False,False,False
2,41.28,0,34496,0,49,6,2020,0.177090,0.659611,False,...,False,True,False,False,False,False,False,False,False,False
3,60.05,1,54767,0,32,6,2020,0.242698,0.063961,False,...,False,False,False,False,False,True,False,False,False,False
4,3.19,1,1126,0,65,6,2020,0.706248,0.867734,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
x_train=train_df_pre.drop('is_fraud',axis=1)
y_train=train_df_pre['is_fraud']
x_test=test_df_pre.drop('is_fraud',axis=1)
y_test=test_df_pre['is_fraud']

In [ ]:
from sklearn.preprocessing import StandardScaler # Import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
import shutil
import os
import pandas as pd

# ... (rest of the existing code) ...

from sklearn.preprocessing import StandardScaler # Import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)
# %%
from sklearn.linear_model import LogisticRegression # Import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report # Import accuracy_score and classification_report

logistic_regression=LogisticRegression()
logistic_regression.fit(x_train,y_train)
y_pred_logistic = logistic_regression.predict(x_test)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_logistic
print("\nClassification Report for Logistic Regression:\n", classification_report(y_test, y_pred_logistic))


Classification Report for Logistic Regression:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719



In [ ]:
DecisionTree=DecisionTreeClassifier()
DecisionTree.fit(x_train,y_train)
y_pred_dt = DecisionTree.predict(x_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_dt
print("\nClassification Report for Decision Tree:\n", classification_report(y_test, y_pred_dt))


Classification Report for Decision Tree:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.55      0.61      0.58      2145

    accuracy                           1.00    555719
   macro avg       0.77      0.80      0.79    555719
weighted avg       1.00      1.00      1.00    555719

